In [1]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import os
import gc
from tqdm import tqdm
import gsw

In [2]:
import sys
sys.path.insert(0, "/data/SO2/SO24/ANDY/SOHI/scripts")
import MEOP

In [3]:
mask = np.load('data1/onshelf_mask.npy')

cube = nc.Dataset("/data/SO2/SO24/ANDY/SOHI/Data/Obs_Climatology/ISMIP6/obs_temperature_1995-2017_8km_x_60m.nc")
ISMIP_XC = cube['lon'][:] 
ISMIP_YC = cube['lat'][:]
ISMIP_RC = cube['z'][:]
ISMIP_THETA = cube['temperature'][:]

cube = nc.Dataset("/data/SO2/SO24/ANDY/SOHI/Data/Obs_Climatology/ISMIP6/obs_salinity_1995-2017_8km_x_60m.nc")
ISMIP_SALT = cube['salinity'][:]

In [4]:
directory = '/data/SO2/SO24/ANDY/SOHI/Data/Models/LLC4320/Monthly_Mean'
LLC4320_Files = os.listdir(directory)
LLC4320_Files.sort()
LLC4320_Files = [f"{directory}/{i}" for i in LLC4320_Files]
hold = LLC4320_Files.pop()
LLC4320_Files.insert(0, hold)
LLC4320_Files

['/data/SO2/SO24/ANDY/SOHI/Data/Models/LLC4320/Monthly_Mean/2012_09.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/LLC4320/Monthly_Mean/2011_10.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/LLC4320/Monthly_Mean/2011_11.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/LLC4320/Monthly_Mean/2011_12.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/LLC4320/Monthly_Mean/2012_01.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/LLC4320/Monthly_Mean/2012_02.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/LLC4320/Monthly_Mean/2012_03.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/LLC4320/Monthly_Mean/2012_04.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/LLC4320/Monthly_Mean/2012_05.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/LLC4320/Monthly_Mean/2012_06.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/LLC4320/Monthly_Mean/2012_07.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/LLC4320/Monthly_Mean/2012_08.nc']

In [5]:
directory = '/data/SO2/SO24/ANDY/SOHI/Data/Models/SOHI/Monthly_Mean'
SOHI_Files = os.listdir(directory)
SOHI_Files.sort()
SOHI_Files = [f"{directory}/{i}" for i in SOHI_Files]
SOHI_Files.pop(0)
SOHI_Files

['/data/SO2/SO24/ANDY/SOHI/Data/Models/SOHI/Monthly_Mean/2005_09.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/SOHI/Monthly_Mean/2005_10.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/SOHI/Monthly_Mean/2005_11.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/SOHI/Monthly_Mean/2005_12.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/SOHI/Monthly_Mean/2006_01.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/SOHI/Monthly_Mean/2006_02.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/SOHI/Monthly_Mean/2006_03.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/SOHI/Monthly_Mean/2006_04.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/SOHI/Monthly_Mean/2006_05.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/SOHI/Monthly_Mean/2006_06.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/SOHI/Monthly_Mean/2006_07.nc',
 '/data/SO2/SO24/ANDY/SOHI/Data/Models/SOHI/Monthly_Mean/2006_08.nc']

In [6]:
SOSE = '/data/SO2/SO24/ANDY/SOHI/Data/Models/SOSE/SOSE_iter100.nc'
ECCO = '/data/SO2/SO24/ANDY/SOHI/Data/Models/ECCO/LLC90_v4r4.nc'

In [7]:
SOSE_t_iters= [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
ECCO_t_iters= [164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175]

In [8]:
#### Months
# Sep : 0
# Oct : 1
# Nov : 2
# Dec : 3
# Jan : 4
# Feb : 5
# Mar : 6
# Apr : 7
# May : 8
# Jun : 9
# Jul : 10
# Aug : 11

months = [9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8]

In [9]:
def find(lat_point, lon_point, lat_field, lon_field):
    if lat_field.ndim == 2:
        i1 = (lon_field - lon_point)**2
        i2 = (lat_field - lat_point)**2
        i3 = i1 + i2
        where = np.where(i3 == i3.min())
        return where[0][0], where[1][0] ## Return row, column
    
    else:
        i1 = np.argmin(abs(lat_field - lat_point))
        i2 = np.argmin(abs(lon_field - lon_point))
        return i1, i2

In [10]:
cube = nc.Dataset(SOHI_Files[0])
SOHI_YC = cube['lat'][:]
SOHI_XC = cube['lon'][:]
SOHI_RC = cube['depth'][:]

cube = nc.Dataset(LLC4320_Files[0])
LLC_YC = cube['lat'][:]
LLC_XC = cube['lon'][:]
LLC_RC = cube['depth'][:]

cube = nc.Dataset(SOSE)
SOSE_YC = cube['lat'][:]
SOSE_XC = cube['lon'][:]
SOSE_RC = cube['depth'][:]

cube = nc.Dataset(ECCO)
ECCO_YC = cube['lat'][:]
ECCO_XC = cube['lon'][:]
ECCO_RC = cube['depth'][:]


In [11]:
def convert_to_insitu(T, S, lat, lon, depth):
    P = abs(depth) * 1023.6 * 9.80665 / 10_000 ## in dbar 
    SA = gsw.SA_from_SP(S, P, lon, lat)
    CT = gsw.CT_from_pt(SA, T)
    IST = gsw.t_from_CT(SA, CT, P)
    
    return IST

In [12]:
index = 11

print('Starting...')

Results = []
## 0 Lat, 1 Lon, 2 Depth, 3 Region_Number, 
## 4 MEOP_T, 5 MEOP_S, 6 SOHI_T, 7 SOHI_S, 8 LLC4320_T, 9 LLC_4320_S, 
## 10 SOSE_T, 11 SOSE_S, 12 ECCO T, 13 ECCO S, 14 ISMIP6 T, 15 ISMIP6 S

# ## Load MEOP
meop = MEOP.load_month(months[index])
meop = meop[meop[:,6] < -60, :]
# #     0: Profile
# #     1: Year
# #     2: Month
# #     3: Day
# #     4: Hour
# #     5: Minute
# #     6: Latitude
# #     7: Longitude
# #     8: Depth (converted from Pressure)
# #     9: Insitu Temperature
# #     10: Practical Salinity


## Load data 
print('Loading LLC4320')
cube = nc.Dataset(LLC4320_Files[index])
LLC_THETA = cube['Theta'][:]
LLC_SALT = cube['Salt'][:]

print('Loading SOHI')
cube = nc.Dataset(SOHI_Files[index])
SOHI_THETA = cube['THETA'][0,:,:,:]
SOHI_SALT = cube['SALT'][0,:,:,:]

print('Loading SOSE')
cube = nc.Dataset(SOSE)
SOSE_THETA = cube['THETA'][SOSE_t_iters[index], :, :, :]
SOSE_SALT = cube['SALT'][SOSE_t_iters[index], :, :, :]

print('Loading ECCO')
cube = nc.Dataset(ECCO)
ECCO_THETA = cube['THETA'][ECCO_t_iters[index], :, :, :]
ECCO_SALT = cube['SALT'][ECCO_t_iters[index], :, :, :]



print("\nInterpolating...")

valid = False
total_profiles = 0 
on_shelf_profiles = 0

for i in tqdm(range(len(meop))):
    
    if meop[i,0] == 1: ## New profile start, recalculate indices
        total_profiles += 1
        iy, ix = find(lat_point = meop[i, 6],
                     lon_point = meop[i, 7],
                     lat_field = ISMIP_YC,
                     lon_field = ISMIP_XC)
        if mask[iy, ix] == 1:
            on_shelf_profiles += 1
            valid = True
            plat = meop[i, 6]
            plon = meop[i, 7]
            pdep = meop[i, 8]
            
            if plon > 44 and plon < 60: # Enderby Land
                reg = 1 
            elif plon > 70 and plon < 82: # Prydz Bay
                reg = 2
            elif plon > 82 and plon < 87: # Leopold and Astrid Coast 
                reg = 3
            elif plon > 92 and plon < 101: # Queen Mary Land 
                reg = 4
            elif plon > 105 and plon < 112: # Knox Coast 
                reg = 5
            elif plon > 135 and plon < 145: # Adelie Coast 
                reg = 6
            elif plon > 160 and plon < 180: ## West Ross
                reg = 7
            elif plon > -120 and plon < -100: ## Amundsen Sea
                reg = 8 
            elif plon > -80 and plon < -70: ## Bellinghausen Sea
                reg = 9 
            elif plon > -65 and plon < -55: ## Peninsula
                reg = 10
            elif plon > -50 and plon < -25: ## Weddell Sea
                reg = 11
            elif plon > -20 and plon < 0: ## Princess Martha Coast 
                reg = 12
            else:
                reg = 0
                
            SOHI_IY, SOHI_IX = find(lat_point = plat,
                     lon_point = plon,
                     lat_field = SOHI_YC,
                     lon_field = SOHI_XC)
            
            LLC_IY, LLC_IX = find(lat_point = plat,
                     lon_point = plon,
                     lat_field = LLC_YC,
                     lon_field = LLC_XC)
            
            SOSE_IY, SOSE_IX = find(lat_point = plat,
                     lon_point = plon,
                     lat_field = SOSE_YC,
                     lon_field = SOSE_XC)
            
            ECCO_IY, ECCO_IX = find(lat_point = plat,
                     lon_point = plon,
                     lat_field = ECCO_YC,
                     lon_field = ECCO_XC)
            
            ISMIP_IY, ISMIP_IX = find(lat_point = plat,
                     lon_point = plon,
                     lat_field = ISMIP_YC,
                     lon_field = ISMIP_XC)
            
        else:
            valid = False 
            
    else: ## Not new profile, update depth
        pdep = meop[i, 8]
            
    if valid:
        SOHI_IZ = np.argmin(abs(SOHI_RC - pdep))
        LLC_IZ = np.argmin(abs(LLC_RC - pdep))
        SOSE_IZ = np.argmin(abs(SOSE_RC - pdep))
        ECCO_IZ = np.argmin(abs(ECCO_RC - pdep))
        ISMIP_IZ = np.argmin(abs(ISMIP_RC - pdep))
        
        SOHI_T = SOHI_THETA[SOHI_IZ, SOHI_IY, SOHI_IX]
        SOHI_S = SOHI_SALT[SOHI_IZ, SOHI_IY, SOHI_IX]
        
        LLC_T = LLC_THETA[LLC_IZ, LLC_IY, LLC_IX]
        LLC_S = LLC_SALT[LLC_IZ, LLC_IY, LLC_IX]
        
        SOSE_T = SOSE_THETA[SOSE_IZ, SOSE_IY, SOSE_IX]
        SOSE_S =  SOSE_SALT[SOSE_IZ, SOSE_IY, SOSE_IX]
        
        ECCO_T = ECCO_THETA[ECCO_IZ, ECCO_IY, ECCO_IX]
        ECCO_S = ECCO_SALT[ECCO_IZ, ECCO_IY, ECCO_IX]
        
        ISMIP_T = ISMIP_THETA[ISMIP_IZ, ISMIP_IY, ISMIP_IX]
        ISMIP_S = ISMIP_SALT[ISMIP_IZ, ISMIP_IY, ISMIP_IX]
        
        SOHI_T = convert_to_insitu(SOHI_T, SOHI_S, plat, plon, pdep)
        LLC_T = convert_to_insitu(LLC_T, LLC_S, plat, plon, pdep)
        SOSE_T = convert_to_insitu(SOSE_T, SOSE_S, plat, plon, pdep)
        ECCO_T = convert_to_insitu(ECCO_T, ECCO_S, plat, plon, pdep)
        ISMIP_T = convert_to_insitu(ISMIP_T, ISMIP_S, plat, plon, pdep)
        
        ## 0 Lat, 1 Lon, 2 Depth, 3 Region_Number, 
        ## 4 MEOP_T, 5 MEOP_S, 6 SOHI_T, 7 SOHI_S, 8 LLC4320_T, 9 LLC_4320_S, 
        ## 10 SOSE_T, 11 SOSE_S, 12 ECCO T, 13 ECCO S, 14 ISMIP6 T, 15 ISMIP6 S
        Results.append([plat, plon, pdep, reg, meop[i, 9], meop[i, 10],
                       SOHI_T, SOHI_S, LLC_T, LLC_S, SOSE_T, SOSE_S, ECCO_T, ECCO_S, ISMIP_T, ISMIP_S])
    else:
        continue
        
fn = [9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8]
Results = np.asarray(Results)
np.save(f'data2/{fn[index]:02d}.npy', Results)

Starting...
Data will be returned as:
    0: Profile
    1: Year
    2: Month
    3: Day
    4: Hour
    5: Minute
    6: Latitude
    7: Longitude
    8: Depth (converted from Pressure)
    9: Insitu Temperature
    10: Practical Salinity

Loading...
Completed! n = 5215381 
Loading LLC4320
Loading SOHI
Loading SOSE
Loading ECCO

Interpolating...


100%|██████████| 653307/653307 [8:33:57<00:00, 21.19it/s]   
